In [4]:
# Работа по  "Фильтрация изображений"
# Нелинейные фильтры. Медианный фильтр
import cv2
import numpy as np
img = cv2.imread('barcode.png',0)
med_img = cv2.medianBlur(img, 9)
cv2.imshow("Original", img)
cv2.imshow("Median Filter", med_img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [12]:
# Нелинейные фильтры. Билатеральный фильтр
import cv2
import numpy as np
img = cv2.imread('barcode.png',0)
bilateral_img = cv2.bilateralFilter(img,11,150,11)
cv2.imshow("Original", img)
cv2.imshow("Bilateral Filter", bilateral_img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [5]:
# Нелинейные фильтры. Сравнение Median и Bilateral
cv2.imshow("Original", img)
cv2.imshow("Median Filter", med_img)
cv2.imshow("Bilateral Filter", bilateral_img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [13]:
# Нелинейные фильтры. Фильтр CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(bilateral_img)
cv2.imshow("Bilateral Filter", bilateral_img)
cv2.imshow("CLAHE", cl1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# Фильтрация изображений
# Линейные фильтры. Фильтр Собеля
import cv2
import numpy as np
scale = 1
delta = 0
ddepth = cv2.CV_16S

img = cv2.imread('sobel_2.png', cv2.IMREAD_GRAYSCALE)

height, width = img.shape[:2]

res = cv2.resize(img,(int(0.5*width), int(0.5*height)), interpolation = cv2.INTER_CUBIC)

res = cv2.bilateralFilter(res,11,75,75)

rows, cols = res.shape
grad_x = cv2.Sobel(res, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
grad_y = cv2.Sobel(res, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
abs_grad_x = cv2.convertScaleAbs(grad_x)
abs_grad_y = cv2.convertScaleAbs(grad_y)
grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

cv2.imshow('Original', res)
cv2.imshow('Sobel horizontal', grad_x)
cv2.imshow('Sobel vertical', grad_y)
cv2.imshow('Sobel', grad)

cv2.waitKey(0)
cv2.destroyAllWindows() 

In [8]:
# Фильтрация изображений
# Линейные фильтры. Фильтр Габора
import numpy as np
import cv2
 
def build_filters():
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 16):
        kern = cv2.getGaborKernel((ksize, ksize), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        kern /= 1.5*kern.sum()
        filters.append(kern)
    return filters
 
def process(img, filters):
    accum = np.zeros_like(img)
    for kern in filters:
        fimg = cv2.filter2D(img, cv2.CV_8UC3, kern)
        np.maximum(accum, fimg, accum)
    return accum

img = cv2.imread('gabor.jpg', 0)
res = cv2.resize(img,(int(0.5*width), int(0.5*height)), interpolation = cv2.INTER_CUBIC)
filters = build_filters()
res1 = process(res, filters)
cv2.imshow('Original', res)
cv2.imshow('Gabor', res1)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [9]:
# Фильтрация изображений
# Линейные фильтры. Конволюция и деконволюция изображений. 
import numpy as np
import cv2

d=10
noise = 10**(-0.1*27)

def defocus_kernel(d, sz=65):
    kern = np.zeros((sz, sz), np.uint8)
    cv2.circle(kern, (sz, sz), d, 255, -1, cv2.LINE_AA, shift=1)
    kern = np.float32(kern) / 255.0
    return kern

img_bw = cv2.imread('wiener.jpeg',0)
img_rgb = cv2.imread('wiener.jpeg', 1)

img_r = np.zeros_like(img_bw)
img_g = np.zeros_like(img_bw)
img_b = np.zeros_like(img_bw)

img_r = img_rgb[..., 0]
img_g = img_rgb[..., 1]
img_b = img_rgb[..., 2]

img_rgb = np.float32(img_rgb)/255.0
img_bw = np.float32(img_bw)/255.0
img_r = np.float32(img_r)/255.0
img_g = np.float32(img_g)/255.0
img_b = np.float32(img_b)/255.0

IMG_R = cv2.dft(img_r, flags=cv2.DFT_COMPLEX_OUTPUT)
IMG_G = cv2.dft(img_g, flags=cv2.DFT_COMPLEX_OUTPUT)
IMG_B = cv2.dft(img_b, flags=cv2.DFT_COMPLEX_OUTPUT)

psf = defocus_kernel(d)
psf /= psf.sum()
psf_pad = np.zeros_like(img_bw)
kh, kw = psf.shape
psf_pad[:kh, :kw] = psf
PSF = cv2.dft(psf_pad, flags=cv2.DFT_COMPLEX_OUTPUT, nonzeroRows = kh)
PSF2 = (PSF**2).sum(-1)
iPSF = PSF / (PSF2 + noise)[...,np.newaxis]

RES_R = cv2.mulSpectrums(IMG_R, iPSF, 0)
RES_G = cv2.mulSpectrums(IMG_G, iPSF, 0)
RES_B = cv2.mulSpectrums(IMG_B, iPSF, 0)

res_r = cv2.idft(RES_R, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )
res_g = cv2.idft(RES_G, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )
res_b = cv2.idft(RES_B, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )

res_rgb = np.zeros_like(img_rgb)
res_rgb[..., 0] = res_r
res_rgb[..., 1] = res_g
res_rgb[..., 2] = res_b

res_rgb = np.roll(res_rgb, -kh//2, 0)
res_rgb = np.roll(res_rgb, -kw//2, 1)

cv2.imshow('Original', img_rgb)
cv2.imshow('Deconvolution', res_rgb)
#cv2.imshow('Original', img)
cv2.waitKey(0)
cv2.destroyAllWindows() 